# Modélisation Deep Learning — Prévision de la demande énergétique

Ce notebook présente la démarche de modélisation avancée par réseaux de neurones (LSTM, GRU, CNN, CNN-LSTM) pour la prévision de la demande énergétique sur les données traitées.

## 1. Imports et configuration

In [ ]:
%pip install python-dotenv tensorflow
from dotenv import load_dotenv
import os
load_dotenv()
os.chdir("..")

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import yaml
import warnings
warnings.filterwarnings('ignore')

from src.models.dl_utils import create_sequences, scale_data
from src.models.dl_models import build_lstm, build_gru, build_cnn, build_cnn_lstm
from src.models.dl_training import train_and_save
from src.models.evaluation import evaluate_regression

## 2. Chargement de la configuration et des données

In [ ]:
def load_config(config_path: str = "config.yaml") -> dict:
    with open(config_path, "r") as f:
        cfg = yaml.safe_load(f)
    return cfg

cfg = load_config("config.yaml")
processed_dir = Path(cfg["paths"]["data"]["processed"])
models_dir    = Path("models/dl")
models_dir.mkdir(exist_ok=True, parents=True)
zones = [p.stem.replace("_processed_hourly","") for p in processed_dir.glob("*_processed_hourly.parquet")]

In [ ]:
print("Zones disponibles :", zones)

## 3. Définition des hyperparamètres

In [ ]:
LOOKBACK = 24  # ex. 24 h
HORIZON  = 1   # 1 pas à prévoir
BATCH    = 32
EPOCHS   = 50

## 4. Entraînement des modèles DL (LSTM, GRU, CNN, CNN-LSTM)

In [ ]:
metrics = []
for zone in zones:
    for horizon in ["daily", "hourly"]:
        file = processed_dir/f"{zone}_processed_{horizon}.parquet"
        if not file.exists():
            print(f"Fichier manquant pour {zone} {horizon}, passage...")
            continue
        df = pd.read_parquet(file)
        feature_cols = [c for c in df.columns if c != "demand"]
        X, y = create_sequences(df, "demand", feature_cols, LOOKBACK, HORIZON)
        split = int(len(X)*0.8)
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]
        X_train, X_val, scaler = scale_data(X_train, X_val)
        input_shape = (LOOKBACK, len(feature_cols))
        models = {
            "LSTM":      build_lstm(input_shape, HORIZON),
            "GRU":       build_gru(input_shape, HORIZON),
            "CNN":       build_cnn(input_shape, HORIZON),
            "CNN-LSTM":  build_cnn_lstm(input_shape, HORIZON)
        }
        for name, model in models.items():
            print(f"Training {name} for {zone} {horizon}...")
            hist, filepath = train_and_save(
                model, X_train, y_train, X_val, y_val,
                models_dir, zone, horizon, name,
                epochs=EPOCHS, batch_size=BATCH
            )
            y_pred = model.predict(X_val)
            y_val_flat = y_val.ravel()
            y_pred_flat = y_pred.ravel()
            score = evaluate_regression(None, None, pd.Series(y_val_flat), pd.Series(y_pred_flat))
            metrics.append({
                "zone": zone,
                "horizon": horizon,
                "model": name,
                **score
            })

## 5. Sauvegarde et visualisation des métriques

In [ ]:
import pandas as pd
metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv("outputs/reports/all_zones_dl_metrics.csv", index=False)
display(metrics_df.head())
metrics_df.groupby(["zone","horizon","model"])[["MAE","RMSE","MAPE"]].mean().unstack().plot.bar(figsize=(12,6))
plt.title("Comparaison des modèles DL par zone et horizon")
plt.ylabel("Score")
plt.show()

# Résumé et prochaines étapes

- Plusieurs architectures de réseaux de neurones ont été entraînées sur chaque zone et chaque horizon.
- Les métriques (MAE, RMSE, MAPE) sont sauvegardées pour chaque modèle.
- Les performances sont visualisées pour comparaison.
- Prochaines étapes : tuning des hyperparamètres, modèles séquentiels avancés, intégration d’exogènes, analyse des résidus, etc.